In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import os
%load_ext autoreload
%autoreload 2

In [ ]:
function_df = pd.read_csv('ubiquitin/ubiquitin.txt')

In [ ]:
function_df['Assay/Protocol'].unique()

So there are actually 4 assays in this dataframe. I will start with 'Relative E1(Uba1) Enzyme Reactivity: with Limiting E1 (200 nM)'

In [ ]:
function_df = function_df[function_df['Assay/Protocol'] == function_df['Assay/Protocol'].unique()[1]]

In [ ]:
function_df['Data'].mean()

In [ ]:
function_df['sequence_len'] = function_df['Sequence'].apply(lambda seq: len(seq)) 
max_len = function_df['sequence_len'].max()

In [ ]:
def sequence_to_aa(seq):
  return list(seq)

In [ ]:
def get_sequence_aa(df):
  df['sequence_aa'] = df['Sequence'].apply(lambda seq:sequence_to_aa(seq))
  df['length'] = df['Sequence'].apply(lambda seq:len(seq))
  sequence_aa = np.array(df['sequence_aa'])
  return sequence_aa

In [ ]:
sequence_aa = get_sequence_aa(function_df)

In [ ]:
# import neccesary tools from Keras
import keras
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sequence_aa)
encoded = tokenizer.texts_to_sequences(sequence_aa)
vocab_len = len(tokenizer.word_index) + 1

In [ ]:
X = np.array(encoded)
y = function_df['Data']

from sklearn.model_selection import train_test_split
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size = 0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size = 0.1)

In [ ]:
from keras.layers import Input, Dense, Embedding, LSTM, Bidirectional, Dropout, BatchNormalization, Softmax, multiply, Lambda, Flatten, Activation, RepeatVector, Permute, LeakyReLU
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.backend as K
from keras.optimizers import Adam, RMSprop

# from fit_one_cycle.clr import LRFinder

In [ ]:
LSTM_size = 10
dense_sizes = [16]
LSTM_dropout = 0.0
n_epochs = 200
adam_lr = [0.001]
initializers = ['he_uniform']

In [ ]:
# https://github.com/keras-team/keras/issues/4962
def get_model(initializer, LSTM_size, dense_size):
    input = Input(shape = [max_len])
    embedding = Embedding(input_dim = vocab_len, 
                          output_dim = 20, 
                          input_length = max_len, 
                          trainable = True)(input)

    activations = Bidirectional(LSTM(LSTM_size, return_sequences = True,
                       dropout = LSTM_dropout,
                       kernel_initializer = initializer))(embedding)

    attention = Dense(1, activation = 'tanh', kernel_initializer = initializer)(activations)
    attention = Flatten()(attention)
    attention = Activation('softmax', name = 'attention_activations')(attention)
    attention = RepeatVector(LSTM_size * 2)(attention)
    attention = Permute([2, 1])(attention)

    sequence_representation = multiply([activations, attention])
    sequence_representation = Lambda(lambda xin: K.sum(xin, axis = -2), output_shape = (LSTM_size * 2,))(sequence_representation)
    
    sequence_representation = Dense(dense_size, kernel_initializer = initializer)(sequence_representation)
    sequence_representation = Dropout(0.2)(sequence_representation)
    sequence_representation = LeakyReLU()(sequence_representation)
    
    # sequence_representation = Dense(8, kernel_initializer = initializer)(sequence_representation)
    # sequence_representation = LeakyReLU()(sequence_representation)
    # sequence_representation = Dropout(0.2)(sequence_representation)

    output = Dense(1, bias_initializer = keras.initializers.Constant(0.7341))(sequence_representation)

    model = Model(inputs = [input], outputs = output)
    return model

In [ ]:
for i in range(len(adam_lr)):
    lr = adam_lr[i]
    for j in range(len(initializers)):
        initializer = initializers[j]
        for k in range(len(dense_sizes)):
            dense_size = dense_sizes[k]
            print(f'adam_lr - {lr} initializer - {initializer}')

            optimizer = Adam(lr = lr)
            model = get_model(initializer, LSTM_size, dense_size)
            model.compile(optimizer = optimizer,
                         loss = 'mean_squared_error')
            # early_stopping = EarlyStopping(patience = 300)

            folder_path = 'ubiquitin/model_and_checkpoints/' + f'LSTM size - {LSTM_size} epochs - {n_epochs} adam_lr {lr} initializer - {initializer}'
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)
            checkpointing = ModelCheckpoint(folder_path + 'weights.{epoch:02d}-{val_loss:.5f}.hdf5',
                                   save_best_only = True)
            history = model.fit(X_train, y_train, validation_data = (X_val, y_val), 
                                batch_size = 128, epochs = n_epochs, verbose = 1,
                                callbacks = [checkpointing])

# save the model summary
#         model_json = model.to_json()
#         with open(folder_path + '.json', 'w') as file:
#             file.write(model_json)
# get the Pearson score
# y_pred = model.predict(X_test)
# y_pred = np.reshape(y_pred, (1094,))

# from scipy.stats import pearsonr, spearmanr
# pearson_score = pearsonr(y_test, y_pred)
# print(f'Pearson correlation - {pearson_score}')
# spearman_score = spearmanr(y_test, y_pred)
# print(f'Spearman correlation - {spearman_score}')

# import pickle
# pickle.dump(pearson_score, open(f'LSTM units - {LSTM_units} epochs - {n_epochs} adam_lr {adam_lr} v3.p', 'wb'))

# visualize the training and validation loss curves
            plt.figure()
            loss = history.history['loss']
            val_loss = history.history['val_loss']
            epoch = np.arange(n_epochs)
            plt.plot(epoch, loss, label = 'loss')
            plt.plot(epoch, val_loss, label = 'val_loss')
            plt.title(f'BiLSTM size - {LSTM_size} dense size - {dense_size} LSTM dropout - {LSTM_dropout} epochs - {n_epochs} adam_lr {lr} initializer - {initializer}')
            plt.legend()
            plt.savefig(folder_path + '.png')

In [ ]:
# get the Pearson score
y_pred = model.predict(X_val)
y_pred = np.reshape(y_pred, (-1,))

from scipy.stats import pearsonr, spearmanr
pearson_score = pearsonr(y_val, y_pred)
print(f'Pearson correlation - {pearson_score}')